Imports

In [49]:
import numpy as np
import cv2
import os
import glob

import matplotlib.pyplot as plt

from cv2 import aruco

from calibration_utils import *

Setting Source directory for camera calibration

In [50]:
root = '/home/eryk-dev/Desktop/inzynieka/Calibration_Code/kalibracja__07_07_2023/'

data_path = root + '/cam2/'

img_ext = "*.png"

img_shape = (1280, 720)

Setting calibration values

In [51]:
board_shape = [11, 7]


# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


Preparing object points

In [52]:
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
parameters =  aruco.DetectorParameters()
arucoDetector = aruco.ArucoDetector(dictionary, parameters)


board = aruco.CharucoBoard(board_shape, 25, 19.0, dictionary)

Flag to display images while collecting data.

In [53]:
display_images = False

Processing Images

In [54]:

images = glob.glob(data_path + img_ext)
images.sort()

print(len(images))

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)

corner_ids = cornerIds(board_shape)

allCorners = []
allIds = []
decimator = 0

#objPoints, imgPoints = [], []


for i, fname in enumerate(images):
    print("[" + str((i+1)) + "/" + str(len(images)) + "]")
    head, tail = os.path.split(fname)

    bgr = cv2.imread(images[i], cv2.IMREAD_UNCHANGED)

    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    corners, ids, rejectedCorners = arucoDetector.detectMarkers(gray)

    if len(corners) > 16:
        for corner in corners:
            cv2.cornerSubPix(gray, corner,
                                winSize = (3,3),
                                zeroZone = (-1,-1),
                                criteria = criteria)
                    
        res2 = aruco.interpolateCornersCharuco(corners, ids, gray, board)
            
        if res2[1] is not None and res2[2] is not None and len(res2[1])>3 and decimator%1==0 and max(ids) <= max(board.getIds()):
            if is_slice_in_list(numpy.squeeze(ids).tolist(), corner_ids): # all corners are detected
                params, board_rot_deg  = get_parameters(corners, numpy.squeeze(ids).tolist(), corner_ids, img_shape, board_shape)
                
                allCorners.append(res2[1])
                allIds.append(res2[2])

                decimator+=1
                    
                print('Chessboard detected ' + tail)

                if display_images :
                    # Draw and display the corners
                    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
                    
                    frame_copy = aruco.drawDetectedMarkers(rgb, corners, ids)
                    
                    f, axarr = plt.subplots(1,1)
                    axarr[0].imshow(frame_copy)
                    plt.show()

        else:
            print("chessboards missing")

    else:
        print("chessboards missing")

print("\nDetected " + str(len(corners)) + " points")

90
[1/90]
Chessboard detected 1.png
[2/90]
Chessboard detected 10.png
[3/90]


Chessboard detected 11.png
[4/90]
Chessboard detected 12.png
[5/90]
Chessboard detected 13.png
[6/90]
Chessboard detected 14.png
[7/90]
Chessboard detected 15.png
[8/90]
Chessboard detected 16.png
[9/90]
Chessboard detected 17.png
[10/90]
Chessboard detected 18.png
[11/90]
Chessboard detected 19.png
[12/90]
Chessboard detected 2.png
[13/90]
Chessboard detected 20.png
[14/90]
Chessboard detected 21.png
[15/90]
Chessboard detected 22.png
[16/90]
Chessboard detected 23.png
[17/90]
Chessboard detected 24.png
[18/90]
Chessboard detected 25.png
[19/90]
Chessboard detected 26.png
[20/90]
Chessboard detected 27.png
[21/90]
Chessboard detected 28.png
[22/90]
Chessboard detected 29.png
[23/90]
Chessboard detected 3.png
[24/90]
Chessboard detected 30.png
[25/90]
Chessboard detected 31.png
[26/90]
Chessboard detected 32.png
[27/90]
Chessboard detected 33.png
[28/90]
Chessboard detected 34.png
[29/90]
Chessboard detected 35.png
[30/90]
Chessboard detected 36.png
[31/90]
Chessboard detected 37.png
[

Preparing for stereo calibration

In [55]:
stereocalib_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 1000, 1e-5)

In [56]:
print("CAMERA CALIBRATION")

cameraMatrixInit = np.array([[ 1000.,    0., img_shape[0] / 2.],
                            [    0., 1000.,  img_shape[1] / 2.],
                            [    0.,    0.,           1.]])

distCoeffsInit = np.zeros((5,1))

flags = (cv2.CALIB_USE_INTRINSIC_GUESS + cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
#flags = (cv2.CALIB_RATIONAL_MODEL)

CAMERA CALIBRATION


Stereo calibration

In [57]:
print("Calculating ....")

imgPoints, objPoints = calculateImgPointsObjPoints(allIds, allCorners, board)

ret, camera_matrix, distortion_coefficients, \
rotation_vectors, translation_vectors = cv2.calibrateCamera(
    objectPoints=imgPoints, 
    imagePoints=objPoints, 
    imageSize=img_shape, 
    cameraMatrix=cameraMatrixInit,
    distCoeffs=distCoeffsInit,
    flags=flags,
    criteria=(cv2.TERM_CRITERIA_EPS & cv2.TERM_CRITERIA_COUNT, 10000, 1e-10))

print("Done")

Calculating ....
Done


Display Error

In [58]:
print("error: {}".format(ret))

error: 0.5748681922871426


Printing results

In [59]:
print('Intrinsic_mtx', camera_matrix)

print('dist', distortion_coefficients)

Intrinsic_mtx [[721.15171888   0.         619.74261322]
 [  0.         721.15171888 357.30179139]
 [  0.           0.           1.        ]]
dist [[ 6.29417639e+00]
 [ 1.24796650e+00]
 [ 1.49557219e-03]
 [-2.58949749e-04]
 [-1.88194984e-01]
 [ 6.08914641e+00]
 [ 1.02078045e+00]
 [ 2.35821293e-01]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]


Saving results

In [60]:
np.savetxt('Intrinsic_mtx_1.txt', camera_matrix)
np.savetxt('dist_1.txt', distortion_coefficients)